In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import image_uris

role = get_execution_role()
sess = sagemaker.Session()

In [2]:
bucket = 'sagemaker-multi-label-test'
prefix = 'ic-multilabel'

s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

In [3]:
train_data = sagemaker.inputs.TrainingInput(
    s3train, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.inputs.TrainingInput(
    s3validation, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [4]:
from sagemaker.image_uris import retrieve, config_for_framework
training_image = retrieve('image-classification', sess.boto_region_name, )
print (training_image)


685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:1


In [5]:
multilabel_ic = sagemaker.estimator.Estimator(training_image,
                                             role, 
                                             instance_count=1, 
                                             instance_type='ml.p3.2xlarge',
                                             output_path=s3_output_location,
                                             sagemaker_session=sess)

In [6]:
multilabel_ic.set_hyperparameters(num_layers=18,
                                image_shape = '3,224,224',
                                num_classes=10,
                                epochs=50,
                                num_training_samples=3457,
                                use_weighted_loss=1,
                                precision_dtype='float32',
                                augmentation_type = 'crop',
                                multi_label=1,
                                top_k=2)

In [8]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from time import gmtime, strftime

tuning_job_name = "imageclassif-job-{}".format(strftime("%d-%H-%M-%S", gmtime()))

hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.00044593590936931976, 0.05, 'Logarithmic'),
                         'mini_batch_size': IntegerParameter(90, 200),
                         'optimizer': CategoricalParameter(['adam', 'nag'])}

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(multilabel_ic,
                            objective_metric_name,
                            hyperparameter_ranges,
                            objective_type='Maximize',
                            max_jobs=10,
                            max_parallel_jobs=1) 

In [ ]:
tuner.fit(data_channels, job_name=tuning_job_name, include_cls_metadata=False)
tuner.wait()

....................................................................................................................................................................................................................................................................................................

In [10]:
best_job = tuner.best_estimator()


2021-03-04 01:16:47 Starting - Preparing the instances for training
2021-03-04 01:16:47 Downloading - Downloading input data
2021-03-04 01:16:47 Training - Training image download completed. Training in progress.
2021-03-04 01:16:47 Uploading - Uploading generated training model
2021-03-04 01:16:47 Completed - Training job completed


In [11]:
ic_classifier = best_job.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-----------------!

In [24]:
import boto3
import botocore
import mxnet as mx
from PIL import Image
import numpy as np

lst = open('val.lst')
s3 = boto3.resource('s3')

bucket = 'pixelscrapper-user-content'
image_dir = 'test-images/'
img_labels = []
img_paths = []

i = 0
for line in lst:
    if i <= 30:
        line = line.split()
        s3_img_path = line[len(line) - 1][1:]
        local_img_path = s3_img_path.split('/')
        local_img_path = local_img_path[len(local_img_path) - 1]
        img_paths.append(image_dir + local_img_path)
        img_labels.append([float(i) for i in line[1:len(line)-1]])
        s3.Bucket(bucket).download_file(s3_img_path, image_dir + local_img_path)
        
        
        img = Image.open(image_dir + local_img_path)
        img = img.convert("RGB")   
        img_array = np.array(img.resize((224, 224)))
        Image.fromarray(img_array).save(image_dir + local_img_path)
        
        
        i = i+1
    else:
        break

In [28]:
import json
import boto3
 
endpoint = 'imageclassif-job-04-00-06-49-008-fa1272-2021-03-04-01-59-50-199' 
runtime = boto3.Session().client('sagemaker-runtime')
tags = ['flower','leaf','star','heart','christmas','template','word art','tag','vintage','illustration']
i = 0
for path in img_paths:
    with open(path, 'rb') as f:
        payload = f.read()
    response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)
    result = json.loads(response['Body'].read().decode())
    print('-------------------------------------')
    print('image:  ' + path)
    j = 0
    print()
    print('actual' + '\t' + 'tag' + '\t' + '\t' + 'response')
    s = 0
    for tag in img_labels[i]:
        s = s+result[j]
        print(str(tag) + '\t' + tags[j] + '\t' + '\t' + str(result[j]))
        j = j+1
    i = i+1
    print(s)

-------------------------------------
image:  test-images/my-bujo-day-blue-12-graphic-tag-number-planner-element-embellishment-green-white.png

actual	tag		response
0.0	flower		0.002540196292102337
0.0	leaf		0.9431027770042419
0.0	star		0.00028306094463914633
0.0	heart		0.003589375875890255
0.0	christmas		0.0006178766489028931
0.0	template		1.4928184100426733e-05
0.0	word art		0.000949955836404115
1.0	tag		0.0001413622812833637
0.0	vintage		0.1128220334649086
0.0	illustration		0.0015701208030804992
1.0656316873355536
-------------------------------------
image:  test-images/seriously-floral-tag07i-graphic-tag-element-vintage-red-yellow-green-pink.png

actual	tag		response
0.0	flower		0.03150792047381401
0.0	leaf		0.027877531945705414
0.0	star		0.0003635817556641996
0.0	heart		9.193750156555325e-05
0.0	christmas		0.052676696330308914
0.0	template		4.864662332693115e-05
0.0	word art		0.0032706407364457846
1.0	tag		0.007840005680918694
1.0	vintage		0.3538884222507477
0.0	illustration		0.2

-------------------------------------
image:  test-images/the-good-life-march-2020-elements-glitter-heart-2-graphic-love-element-cork-white-brown.png

actual	tag		response
0.0	flower		5.3327479690779e-05
0.0	leaf		3.0516574042849243e-05
0.0	star		0.01913408376276493
1.0	heart		0.4560718536376953
0.0	christmas		2.174217797801248e-06
0.0	template		0.0008975500822998583
0.0	word art		0.002011618809774518
0.0	tag		0.026959672570228577
0.0	vintage		0.009287350811064243
0.0	illustration		0.02599763683974743
0.5404457847851063
-------------------------------------
image:  test-images/sent-template-video-game-valentine-stamp-graphic-element-computer-pixels-games-teen-boy-love-quotes.png

actual	tag		response
0.0	flower		0.006798880640417337
0.0	leaf		0.011343127116560936
0.0	star		0.6640644073486328
0.0	heart		0.002715318463742733
0.0	christmas		0.2349625825881958
1.0	template		0.9677658677101135
0.0	word art		0.9024987816810608
0.0	tag		0.05031498521566391
0.0	vintage		0.0006143121281638741
0

-------------------------------------
image:  test-images/my-bujo-day-blue-12-graphic-tag-number-planner-element-embellishment-green-white.png

actual	response
0.0	0.0006384921725839376
0.0	0.9138441681861877
0.0	0.0004088479618076235
0.0	0.004992966074496508
0.0	0.0001976013445528224
0.0	9.837906873144675e-06
0.0	0.0008342984947375953
tag	9.482325549470261e-05
0.0	0.29560819268226624
0.0	0.0011613955721259117
-------------------------------------
image:  test-images/seriously-floral-tag07i-graphic-tag-element-vintage-red-yellow-green-pink.png

actual	response
0.0	0.014177469536662102
0.0	0.013518800027668476
0.0	0.0006192461005412042
0.0	4.2091145587619394e-05
0.0	0.12789542973041534
0.0	3.777773235924542e-05
0.0	0.0037602137308567762
tag	0.01822315901517868
vintage	0.18592289090156555
0.0	0.1229759007692337
-------------------------------------
image:  test-images/summer-splash-illustrations-1-popsicle-graphic-element-illustration-beach-pool-orange-yellow-green.png

actual	response
0

-------------------------------------
image:  test-images/textured-grunge-star-08-asset-shape-cutout-stamp-embellishment-texture-distressed-white-gray.png

actual	response
0.0	0.9795093536376953
0.0	0.00486045191064477
star	0.13602083921432495
0.0	2.701589210118982e-07
0.0	0.00017346919048577547
0.0	0.09785807877779007
0.0	0.0005529433256015182
0.0	0.00016199852689169347
0.0	0.00013510763528756797
0.0	1.3609246174439704e-08
-------------------------------------
image:  test-images/vintage-gifts-template-graphic-illustration-element-stamp-printable-christmas-black.png

actual	response
0.0	3.060748895222787e-06
0.0	0.0012575339060276747
0.0	2.225097887276206e-05
0.0	1.9509971025399864e-05
christmas	0.0038989665918052197
template	0.9980186223983765
0.0	0.5461789965629578
0.0	0.02695574425160885
vintage	0.003237880300730467
illustration	0.9754362106323242
-------------------------------------
image:  test-images/back-to-basics-everyday-label-170-graphic-tab-tag-wordart-element-details-gold

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (413) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>
". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/imageclassif-job-04-00-06-49-008-fa1272-2021-03-04-01-59-50-199 in account 466363126778 for more information.